In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, Activation
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import seaborn as sns
import time

### Load data

In mysql db in my local environment but loaded using csvs for portability

In [392]:
#district data
districts_df = pd.read_csv('../table_csvs/districts_table.csv').set_index(['District_Id'])

#budget data
functions_df = pd.read_csv('../table_csvs/functions_table.csv').set_index(['District_Id','Year'])
programs_df = pd.read_csv('../table_csvs/programs_table.csv').set_index(['District_Id','Year'])
revenue_df = pd.read_csv('../table_csvs/revenue_table.csv').set_index(['District_Id','Year'])

#school data
teachers_df = pd.read_csv('../table_csvs/teachers_table.csv').set_index(['District_Id','Year'])
enrollment_df = pd.read_csv('../table_csvs/enrollment_table.csv').set_index(['District_Id','Year'])
classes_df = pd.read_csv('../table_csvs/classes_table.csv').set_index(['District_Id','Year'])

#evaluation data
test_scores_df = pd.read_csv('../table_csvs/test_scores_table.csv').set_index(['District_Id','Year'])
dropout_rates_df = pd.read_csv('../table_csvs/dropout_rates_table.csv').set_index(['District_Id','Year'])

#Uncomment one at a time to see columns and different metrics of each
# functions_df.describe().T
# programs_df.describe().T
# revenue_df.describe().T

# teachers_df.describe().T
# enrollment_df.describe().T
# classes_df.describe().T

# test_scores_df.describe().T
# dropout_rates_df.describe().T

### Preprocess Data

Create new df for function expenditure per student 

In [3]:
funct_per_student = pd.concat([functions_df, enrollment_df], axis=1)
funct_per_student = funct_per_student.div(funct_per_student['Fall_Enrollment'],axis=0).drop(['Fall_Enrollment'],axis=1)
funct_per_student.head()

Instruction_All_Funds  Instruction_General_Funds  \
District_Id Year                                                     
1902        2005            4092.806507                3783.885274   
            2006            4229.204467                3854.426117   
            2007            4644.550088                4307.441125   
            2008            7083.765517                4727.293103   
            2009            7393.742424                4988.651515   

                  Instructional_Res_Media_All_Funds  \
District_Id Year                                      
1902        2005                         103.359589   
            2006                         137.572165   
            2007                         136.209139   
            2008                         137.248276   
            2009                         142.424242   

                  Instructional_Res_Media_General_Funds  \
District_Id Year                                          
1902        2005                              81.119863   
            2006                             135.142612   
            2007                             133.720562   
            2008                              93.820690   
            2009                             101.846801   

                  Curriculum_Staff_Develop_All_Funds  \
District_Id Year                                       
1902        2005                            1.919521   
            2006                            7.309278   
            2007                            8.732865   
            2008                           29.255172   
            2009                           19.149832   

                  Curriculum_Staff_Develop_General_Funds  \
District_Id Year                                           
1902        2005                                1.077055   
            2006                                2.637457   
            2007                                1.441125   
            2008                                7.881034   
            2009                                2.149832   

                  Instructional_Leadership_All_Funds  \
District_Id Year                                       
1902        2005                            9.458904   
            2006                           27.673540   
            2007                            7.029877   
            2008                          522.131034   
            2009                          366.493266   

                  Instructional_Leadership_General_Funds  \
District_Id Year                                           
1902        2005                                     0.0   
            2006                                     0.0   
            2007                                     0.0   
            2008                                     0.0   
            2009                                     0.0   

                  School_Administration_All_Funds  \
District_Id Year                                    
1902        2005                       409.648973   
            2006                       420.606529   
            2007                       448.697715   
            2008                       445.346552   
            2009                       446.126263   

                  School_Administration_General_Funds  ...  \
District_Id Year                                       ...   
1902        2005                           407.044521  ...   
            2006                           417.919244  ...   
            2007                           448.697715  ...   
            2008                           445.346552  ...   
            2009                           446.126263  ...   

                  Plant_Maintenance_Operation_All_Funds  \
District_Id Year                                          
1902        2005                             699.385274   
            2006                             853.317869   
            2007                             950.420035 

Create new df for program expenditure per student 

In [4]:
program_per_student = pd.concat([programs_df, enrollment_df], axis=1)
program_per_student = program_per_student.div(program_per_student['Fall_Enrollment'],axis=0).drop(['Fall_Enrollment'],axis=1)
program_per_student.head()

Regular_All_Funds  Regular_General_Funds  \
District_Id Year                                             
1902        2005        3859.306507            3790.794521   
            2006        3834.027491            3762.843643   
            2007        4186.446397            4162.520211   
            2008        4480.765517            4426.168966   
            2009        4782.831650            4702.267677   

                  Gifted_Talented_All_Funds  Gifted_Talented_General_Funds  \
District_Id Year                                                             
1902        2005                  17.511986                      17.347603   
            2006                  17.798969                      17.634021   
            2007                  17.249561                      17.249561   
            2008                  17.682759                      17.682759   
            2009                  17.259259                      17.259259   

                  Career_Technology_All_Funds  \
District_Id Year                                
1902        2005                   161.032534   
            2006                   168.982818   
            2007                   201.260105   
            2008                   194.972414   
            2009                   224.781145   

                  Career_Technology_General_Funds  \
District_Id Year                                    
1902        2005                       159.804795   
            2006                       167.675258   
            2007                       201.260105   
            2008                       194.972414   
            2009                       224.781145   

                  Students_With_Disabilities_All_Funds  \
District_Id Year                                         
1902        2005                            510.321918   
            2006                            588.969072   
            2007                            630.653779   
            2008                           4636.396552   
            2009                           4313.175084   

                  Students_With_Disabilities_General_Funds  \
District_Id Year                                             
1902        2005                                502.470890   
            2006                                582.180412   
            2007                                630.653779   
            2008                                672.124138   
            2009                                633.122896   

                  Compensatory_Education_All_Funds  \
District_Id Year                                     
1902        2005                        371.265411   
            2006                        525.262887   
            2007                        551.328647   
            2008                        585.068966   
            2009                        614.392256   

                  Compensatory_Education_General_Funds  ...  \
District_Id Year                                        ...   
1902        2005                            104.974315  ...   
            2006                            193.261168  ...   
            2007                            221.335677  ...   
            2008                            310.767241  ...   
            2009                            306.365320  ...   

                  Pre_K_Regular_All_Funds  Pre_K_Regular_General_Funds  \
District_Id Year                                                         
1902        2005                      0.0                          0.0   
            2006                      0.0                          0.0   
            2007                      0.0                          0.0   
            2008                      0.0                          0.0   
            2009                      0.0                          0.0   

                  Pre_K_Bilingual_All_Funds  Pre_K_Bilingual_General_Funds  \
District_Id Year                                            

Create new df for program expenditure per student

In [579]:
revenue_per_student = pd.concat([revenue_df, enrollment_df], axis=1)
revenue_per_student = revenue_per_student.div(revenue_per_student['Fall_Enrollment'],axis=0).drop(['Fall_Enrollment'],axis=1)
revenue_per_student.head()

Total_Revenue_All_Funds  Total_Revenue_General_Funds  \
District_Id Year                                                         
1902        2005              9583.652397                  8857.416096   
            2006              8880.910653                  8057.941581   
            2007             10726.945518                  9933.328647   
            2008             16678.003448                 10166.601724   
            2009             13931.471380                  9493.080808   

                  Total_Federal_Revenue_All_Funds  \
District_Id Year                                    
1902        2005                       410.268836   
            2006                       507.532646   
            2007                       439.662566   
            2008                      3193.484483   
            2009                      2887.185185   

                  Total_Federal_Revenue_General_Funds  \
District_Id Year                                        
1902        2005                                  0.0   
            2006                                  0.0   
            2007                                  0.0   
            2008                                  0.0   
            2009                                  0.0   

                  Total_State_Revenue_All_Funds  \
District_Id Year                                  
1902        2005                    1341.828767   
            2006                    1004.886598   
            2007                    2067.281195   
            2008                    4051.950000   
            2009                    4163.927609   

                  Total_State_Revenue_General_Funds  \
District_Id Year                                      
1902        2005                        1234.905822   
            2006                         898.350515   
            2007                        1958.135325   
            2008                        3181.434483   
            2009                        3283.112795   

                  Total_Local_Revenue_All_Funds  \
District_Id Year                                  
1902        2005                    6462.398973   
            2006                    6604.077320   
            2007                    7245.258348   
            2008                    6278.191379   
            2009                    5587.818182   

                  Total_Local_Revenue_General_Funds  
District_Id Year                                     
1902        2005                        6462.398973  
            2006                        6604.077320  
            2007                        7245.258348  
            2008                        6278.191379  
            2009                        5587.818182

Combine budget data and test score data into one df
- Avg_Act has the most present values which is why it is used here
- Could have used program_per_student data here but funct_per_student provides a more general view of where money is spent

In [927]:
#Test score metric used for analysis
act_test_score_column = 'Avg_Sat'
act_hi_lo = [750,1200]

sat_test_score_column = 'Avg_Act'
sat_hi_lo = [15,25]

#Combine into one df
act_dataset = pd.concat([revenue_per_student, program_per_student, funct_per_student, test_scores_df[act_test_score_column].to_frame()], axis=1).reset_index()
sat_dataset = pd.concat([revenue_per_student, program_per_student, funct_per_student, test_scores_df[sat_test_score_column].to_frame()], axis=1).reset_index()

Clean up the data

In [928]:
def clean_dataset(dataset):
    dataset = dataset[dataset.Year > 2004] #No budget data for prior to 2004

    #get only rows with data for all years 2005-2019
    for dist in dataset.District_Id.unique():
        if len(dataset[dataset.District_Id == dist].Year.unique()) != len(range(2005,2020)):
            dataset = dataset[dataset.District_Id != dist]

    #remove charter schools
    non_charter_districts = districts_df[districts_df['Charter_Flag'] == 0].reset_index()
    dataset = pd.merge(non_charter_districts, dataset, on='District_Id', how='inner').drop(['Charter_Flag', 'District_Name'],axis=1)

    return dataset

clean_act_dataset = clean_dataset(act_dataset)
clean_sat_dataset = clean_dataset(sat_dataset)

Choose which funds to look at:
- General Funds are funds that districts can spend in categories they choose
- All funds include general funds but also inlcude funds that have to be used for specific purposes

In [929]:
filtered_act_dataset = clean_act_dataset.filter(regex='(.*_Funds$)|(Avg_.*)|(District_Id)|(Year)')
filtered_sat_dataset = clean_sat_dataset.filter(regex='(.*_Funds$)|(Avg_.*)|(District_Id)|(Year)')
# filtered_dataset = dataset.filter(regex='(.*_All_Funds$)|(Avg_Act)|(District_Id)|(Year)')

#remove any other features
# filtered_dataset = filtered_dataset.drop(['Social_Work_Services_General_Funds','Food_General_Funds','Community_Services_General_Funds','Health_Services_General_Funds', 'Transportation_General_Funds', 'Plant_Maintenance_Operation_General_Funds'], axis=1)
# filtered_dataset = filtered_dataset[["District_Id","Year","Compensatory_Education_General_Funds","Avg_Act"]]

# filtered_dataset

Break dataset into windows so we can attempt to predict the next years test score based on previous results and budget data features

In [931]:
def split_windows(df, window_size=4):
    inputs = np.empty((0, window_size, len(df.columns)))
    labels = np.array([])
    
    for i in range(len(df.Year.unique()) - window_size + 1):
        start_year = 2005
        
        window_df = df.loc[df.Year >= start_year + i].loc[df.Year < start_year + i + window_size]
        ids = len(window_df.District_Id.unique())
        window_array = np.array(window_df)
        
        window_inputs = np.array(np.array_split(window_array, ids))
        
        inputs = np.concatenate((inputs, window_inputs),axis=0)
    
    #remove data points that have nan in score column
    inputs = inputs[np.sum(np.isnan(inputs[:,:,-1]),axis=1) == 0]
    
    return inputs

def get_labels(data):
    labels = data[:,-1,-1].copy()
    data[:,-1,-1] = 1
    
    return labels

def get_train_test(inputs, splits, hi_lo, hi_lo_duplicates=2):
    scores = label_scaler.inverse_transform(inputs[:,-1,-1].reshape(1,-1))
    
    hi = inputs[(scores > hi_lo[1]).reshape(inputs[:,-1,-1].shape),:,:]
    lo = inputs[(scores < hi_lo[0]).reshape(inputs[:,-1,-1].shape),:,:]
    mid = inputs[((scores <= hi_lo[1]) * (scores >= hi_lo[0])).reshape(inputs[:,-1,-1].shape),:,:]
    
    np.random.shuffle(hi)
    np.random.shuffle(lo)
    np.random.shuffle(mid)
    
    hi_splits = (splits * hi.shape[0]).astype(int)
    train_hi = hi[:hi_splits[0],:,:]
    test_hi = hi[hi_splits[0]:hi_splits[1],:,:]

    mid_splits = (splits * mid.shape[0]).astype(int)
    train_mid = mid[:mid_splits[0],:,:]
    test_mid = mid[mid_splits[0]:mid_splits[1],:,:]

    lo_splits = (splits * lo.shape[0]).astype(int)
    train_lo = lo[:lo_splits[0],:,:]
    test_lo = lo[lo_splits[0]:lo_splits[1],:,:]
    
    test = np.concatenate((test_hi, test_mid, test_lo))
    
    train = train_mid
    for _ in range(hi_lo_duplicates):
        train = np.concatenate((train,train_hi,train_lo))
    
    return train, test
    
def normalize_data(filtered_dataset):
    #Normalize data to range 0-1
    label_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_labels = label_scaler.fit_transform(np.array(filtered_dataset)[:,-1].reshape(-1,1))

    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_features = feature_scaler.fit_transform(np.array(filtered_dataset)[:,2:-1])

    scaled_dataset = pd.DataFrame(np.hstack((np.array(filtered_dataset)[:,:2], scaled_features, scaled_labels)), columns=filtered_dataset.columns)
    
    return scaled_dataset, label_scaler, feature_scaler

scaled_act_dataset, act_label_scaler, act_feature_scaler = normalize_data(filtered_act_dataset)
scaled_sat_dataset, sat_label_scaler, sat_feature_scaler = normalize_data(filtered_sat_dataset)

window = 4
splits = np.array([.8,1])

act_inputs = split_windows(scaled_act_dataset, window)
sat_inputs = split_windows(scaled_sat_dataset, window)

act_train, act_test = get_train_test(act_inputs, splits, act_hi_lo,1)
sat_train, sat_test = get_train_test(sat_inputs, splits, sat_hi_lo,1)

np.random.shuffle(act_train)
np.random.shuffle(act_test)

np.random.shuffle(sat_train)
np.random.shuffle(sat_test)

act_train_inputs_district_and_year = act_train
act_train_labels = get_labels(train)

sat_train_inputs_district_and_year = sat_train
sat_train_labels = get_labels(train)

act_test_inputs_district_and_year = act_test
act_test_labels = get_labels(act_test)

sat_test_inputs_district_and_year = sat_test
sat_test_labels = get_labels(sat_test)

### Build the Model

Split train and test data

In [932]:
#Remove year and district id
act_train_inputs = act_train_inputs_district_and_year[:,:,2:]
act_test_inputs = act_test_inputs_district_and_year[:,:,2:]

sat_train_inputs = sat_train_inputs_district_and_year[:,:,2:]
sat_test_inputs = sat_test_inputs_district_and_year[:,:,2:]

def get_individual_features(train_inputs, test_inputs):
    train_inputs_list = []
    test_inputs_list = []
    
    for i in range(train_inputs.shape[2]):
        train_inputs_list.append(train_inputs[:,:,i].reshape(train_inputs.shape[0],window,1))
        test_inputs_list.append(test_inputs[:,:,i].reshape(test_inputs.shape[0],window,1))
    
    return train_inputs_list, test_inputs_list

#Create inputs that just consist of past scores
act_scores_train_inputs = act_train_inputs[:,:,-1].reshape(act_train_inputs.shape[0],window,1)
act_scores_test_inputs = act_test_inputs[:,:,-1].reshape(act_test_inputs.shape[0],window,1)

sat_scores_train_inputs = sat_train_inputs[:,:,-1].reshape(sat_train_inputs.shape[0],window,1)
sat_scores_test_inputs = sat_test_inputs[:,:,-1].reshape(sat_test_inputs.shape[0],window,1)

#Create inputs that just consist of just budget data
act_budget_train_inputs = act_train_inputs[:,:,(2,4,5,16,17,42,43,52,53)]
act_budget_test_inputs = act_test_inputs[:,:,(2,4,5,16,17,42,43,52,53)]

sat_budget_train_inputs = sat_train_inputs[:,:,(2,4,5,16,17,42,43,52,53)]
sat_budget_test_inputs = sat_test_inputs[:,:,(2,4,5,16,17,42,43,52,53)]

act_train_inputs_list, act_test_inputs_list = get_individual_features(act_train_inputs, act_test_inputs)
sat_train_inputs_list, sat_test_inputs_list = get_individual_features(sat_train_inputs, sat_test_inputs)

# np.sum(label_scaler.inverse_transform(train_labels.reshape(1,-1)) < 15)
# hi_lo

Create model architecture

In [933]:
EPOCHS = 6
BATCH_SIZE = 32

def create_model(window, features):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units = 64, return_sequences = True, input_shape = (window, features)))
    model.add(Dropout(0.2))

#     model.add(tf.keras.layers.LSTM(units = 64, return_sequences = True, input_shape = (window, features)))
#     model.add(Dropout(0.2))
    
#     model.add(tf.keras.layers.LSTM(units = 64, return_sequences = True, input_shape = (window, features)))
#     model.add(Dropout(0.2))

#     model.add(tf.keras.layers.LSTM(units = 64, return_sequences = True, input_shape = (window, features)))
#     model.add(Dropout(0.2))

    model.add(tf.keras.layers.LSTM(units = 64, return_sequences = False))
    model.add(Dropout(0.2))

    model.add(Dense(units = 1))
    
    model.compile(optimizer = tf.optimizers.Adam(), loss = tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanAbsoluteError()])
    
    return model

def fit_predict(model, name, train_data, train_labels, test_data, test_labels, hi_lo):
    model.fit(train_data, train_labels, epochs = EPOCHS, batch_size=BATCH_SIZE)
    
    predictions = model.predict(test_data)
    predictions = label_scaler.inverse_transform(predictions)

    actual = label_scaler.inverse_transform(test_labels.reshape(1,-1))

    comp = pd.DataFrame(np.hstack((actual.T, predictions)),columns=['actual','predicted'])
    
    high = np.array(comp[comp['actual'] > hi_lo[1]])
    mid = np.array(comp[comp['actual'] >= hi_lo[0]][comp['actual'] <= hi_lo[1]])
    low = np.array(comp[comp['actual'] < hi_lo[0]])
    tot = np.array(comp)

    Total_MSE = np.sqrt(np.mean((tot[:,0] - tot[:,1])**2))
    High_MSE = np.sqrt(np.mean((high[:,0] - high[:,1])**2))
    Mid_MSE = np.sqrt(np.mean((mid[:,0] - mid[:,1])**2))
    Low_MSE = np.sqrt(np.mean((low[:,0] - low[:,1])**2))
    

    print(comp.head(5))
    print(comp[comp['actual'] > hi_lo[1]].head(5))
    print(comp[comp['actual'] < hi_lo[0]].head(5))
    
    return pd.DataFrame(data=[[Total_MSE, High_MSE, Mid_MSE, Low_MSE]], index=[name], columns=['Total MSE','High MSE','Mid MSE','Low MSE'])

Train for all feature training sets

In [543]:
results_by_feature = pd.DataFrame(columns=['Total MSE','High MSE','Mid MSE','Low MSE'])

for i in range(len(train_inputs_list)):
    model = create_model(window, train_inputs_list[i].shape[2])
    curr_res = fit_predict(model, str(i), train_inputs_list[i], train_labels, test_inputs_list[i], test_labels)
    results_by_feature = pd.concat([results_by_feature, curr_res])

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
138/138 [==============================] - 1s 6ms/step - loss: 0.0297 - mean_absolute_error: 0.1251
Epoch 2/5
138/138 [==============================] - 1s 6ms/step - loss: 0.0098 - mean_absolute_error: 0.0781
Epoch 3/5
138/138 [==============================] - 1s 6ms/step - loss: 0.0098 - mean_absolute_error: 0.0780
Epoch 4/5
138/138 [==============================] - 1s 6m

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


   actual  predicted
0    17.7  19.743265
1    22.9  19.780870
2    19.7  19.808226
3    22.6  19.795820
4    19.2  19.559723
     actual  predicted
24     28.5  19.729845
30     26.0  19.787525
163    25.5  19.792707
185    25.4  19.771172
333    25.5  19.751284
     actual  predicted
119    14.8  19.579615
182    14.6  19.784492
737    14.8  19.548662
904    14.6  19.735521
997    14.0  19.744663
Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.expe

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


   actual  predicted
0    17.7  20.196779
1    22.9  20.194601
2    19.7  20.195425
3    22.6  20.194094
4    19.2  20.197735
     actual  predicted
24     28.5  20.194599
30     26.0  20.193987
163    25.5  20.195759
185    25.4  20.194439
333    25.5  20.193916
     actual  predicted
119    14.8  20.201468
182    14.6  20.194916
737    14.8  20.200994
904    14.6  20.195854
997    14.0  20.196342
Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.expe

...

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


   actual  predicted
0    17.7  19.581707
1    22.9  20.262606
2    19.7  20.035206
3    22.6  20.085279
4    19.2  19.495348
     actual  predicted
24     28.5  20.053831
30     26.0  20.186548
163    25.5  20.165804
185    25.4  20.124308
333    25.5  20.256449
     actual  predicted
119    14.8  19.336823
182    14.6  19.983658
737    14.8  19.242218
904    14.6  19.806925
997    14.0  19.608322
Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.expe

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


   actual  predicted
0    17.7  19.925104
1    22.9  20.041868
2    19.7  19.986605
3    22.6  20.037167
4    19.2  19.899822
     actual  predicted
24     28.5  19.999815
30     26.0  20.026794
163    25.5  20.013056
185    25.4  20.011562
333    25.5  20.033449
     actual  predicted
119    14.8  19.867178
182    14.6  19.994707
737    14.8  19.840084
904    14.6  19.957201
997    14.0  19.922405
Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.expe

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [548]:
results_by_feature[60:]#2,4,5,16,17,42,43,52,53

,Total MSE,High MSE,Mid MSE,Low MSE
60,2.026706,6.417668,1.901727,5.142687
61,2.045103,6.669615,1.915653,4.934843
62,2.038321,6.345209,1.912307,5.457844
63,2.040415,6.241316,1.916386,5.571196
64,2.054030,6.678010,1.921953,5.165069
65,2.047786,6.474777,1.919454,5.366653
66,1.996257,6.439503,1.867393,5.184419
67,2.032283,6.390210,1.904796,5.419618
68,1.255124,2.265451,1.221506,3.290243


Run model with all data

In [921]:
real_model = create_model(window,train_inputs.shape[2])


real_results = fit_predict(real_model, 'Real', train_inputs, train_labels, test_inputs, test_labels)

Epoch 1/6
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
236/236 [==============================] - 1s 5ms/step - loss: 0.0096 - mean_absolute_error: 0.0703
Epoch 2/6
236/236 [==============================] - 1s 5ms/step - loss: 0.0050 - mean_absolute_error: 0.0541
Epoch 3/6
236/236 [==============================] - 1s 5ms/step - loss: 0.0046 - mean_absolute_error: 0.0518
Epoch 4/6
236/236 [==============================] - 1s 5m

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Run model with garbage data

In [922]:
garbage_model = create_model(window, train_inputs.shape[2])

rand_data = np.random.rand(train_inputs.shape[0],train_inputs.shape[1],train_inputs.shape[2])
garbage_results = fit_predict(garbage_model, 'Garbage', rand_data, train_labels, test_inputs, test_labels)

Epoch 1/6
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
236/236 [==============================] - 1s 5ms/step - loss: 0.0145 - mean_absolute_error: 0.0927
Epoch 2/6
236/236 [==============================] - 1s 5ms/step - loss: 0.0111 - mean_absolute_error: 0.0839
Epoch 3/6
236/236 [==============================] - 1s 5ms/step - loss: 0.0106 - mean_absolute_error: 0.0815
Epoch 4/6
236/236 [==============================] - 1s 5m

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Run model with score data only

In [923]:
scores_only_model = create_model(window,1)

scores_results = fit_predict(scores_only_model, 'Scores_Only', scores_train_inputs, train_labels, scores_test_inputs, test_labels)

Epoch 1/6
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
236/236 [==============================] - 1s 5ms/step - loss: 0.0108 - mean_absolute_error: 0.0739
Epoch 2/6
236/236 [==============================] - 1s 5ms/step - loss: 0.0049 - mean_absolute_error: 0.0534
Epoch 3/6
236/236 [==============================] - 1s 5ms/step - loss: 0.0047 - mean_absolute_error: 0.0523
Epoch 4/6
236/236 [==============================] - 1s 5m

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Run model with budget data only

In [924]:
budget_only_model = create_model(window, budget_train_inputs.shape[2])

budget_results = fit_predict(budget_only_model, 'Budget_Only', budget_train_inputs, train_labels, budget_test_inputs, test_labels)

Epoch 1/6
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
236/236 [==============================] - 1s 5ms/step - loss: 0.0156 - mean_absolute_error: 0.0905
Epoch 2/6
236/236 [==============================] - 1s 5ms/step - loss: 0.0072 - mean_absolute_error: 0.0666
Epoch 3/6
236/236 [==============================] - 1s 5ms/step - loss: 0.0069 - mean_absolute_error: 0.0650
Epoch 4/6
236/236 [==============================] - 1s 5m

/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [925]:
results = pd.concat([real_results, garbage_results, scores_results, budget_results])
results

,Total MSE,High MSE,Mid MSE,Low MSE
Real,1.217532,2.793552,1.182647,2.729701
Garbage,2.643509,7.724277,2.538802,3.944886
Scores_Only,1.218986,2.344857,1.188030,3.308442
Budget_Only,1.551374,3.482391,1.504673,3.910776


In [881]:
results = pd.concat([real_results, garbage_results, scores_results, budget_results])
results

,Total MSE,High MSE,Mid MSE,Low MSE
Real,1.233153,3.046572,1.182257,2.787623
Garbage,1.951714,6.116827,1.845086,5.274192
Scores_Only,1.239441,2.062198,1.209808,3.970565
Budget_Only,1.482804,3.157045,1.444781,3.635425


In [431]:
results = pd.concat([real_results, garbage_results, scores_results, budget_results])
results

,Total MSE,High MSE,Mid MSE,Low MSE
Real,70.854165,142.277616,67.969684,196.812780
Garbage,126.591567,307.195181,118.743303,274.228522
Scores_Only,60.731685,122.581799,58.183514,181.180477
Budget_Only,111.874227,244.001639,106.606308,209.913525
